In [1]:
import pyxdf
import os
import numpy as np
from scipy import stats
from pathlib import Path
import sys
import datetime

from utilities.utl import find_lsl_stream, find_nearest
from utilities.pupil_prep import *
from utilities.SingleSubjectData import SingleSubjectArcherRep

In [19]:
# (import) helper functions
dir_script = Path("__file__").parent.absolute()
dir_rawdata = Path.joinpath(dir_script.parent.absolute(), "04_data","00_raw")
print(dir_script)

C:\Users\User\Desktop\kiel\tremor_feedback_jw\05_scripts


In [24]:
f_list = os.listdir(DATARAWPATH)
str_match = 'archer'
fnms = [s for s in f_list if str_match in s]
f = fnms[12]
print(f"Working on {f.split('_')[0]}")

Working on p202


In [26]:
# init class and load streams
sub = SingleSubjectArcherRep()
sub.load_data(dir_rawdata,f)

streams, _ = pyxdf.load_xdf(os.path.join(DATARAWPATH,f))
fsr  = find_lsl_stream(streams, 'HX711')
mrk = find_lsl_stream(streams, 'PsychoPyMarkers')
eye = find_lsl_stream(streams, 'pupil_capture')

Max force is 4625 something


In [27]:
sub_id = f.split('_')[0]
fname = f
date = datetime.datetime.fromtimestamp(int(float(Path(os.path.join(DATARAWPATH,f)).stat().st_ctime))).date()
time = datetime.datetime.fromtimestamp(int(float(Path(os.path.join(DATARAWPATH,f)).stat().st_ctime))).time()

In [38]:
try:
    nms_mrk = [nm_mrk for mrk_ in mrk["time_series"] for nm_mrk in mrk_]
    max_f = float([nm for nm in nms_mrk if nm.startswith('max_force')][0].split('_')[2])
    print(f"Max force is {max_f:.0f}")
except:
    max_f = 0
    print("No max force found")

Max force is 4625


In [43]:
# epoch data
idxs_eps_end = [i for i,nm in enumerate(nms_mrk) if 'end_trial' in nm]
times_eps_end = mrk['time_stamps'][idxs_eps_end]

print(f"{len(idxs_eps_end)}/14 epochs found")

14/14 epochs found


In [44]:
# eye tracking data
nms_label_pupillab =[nm_lb['label'][0] for nm_lb in eye['info']['desc'][0]['channels'][0]['channel']]

idx_diam0_3d =  nms_label_pupillab.index('diameter0_3d')
idx_diam1_3d =  nms_label_pupillab.index('diameter1_3d')
idx_conf =  nms_label_pupillab.index('confidence')

# (1) confidence threshold
idx_conf, _ = confidence_threshold(eye['time_series'][:,idx_diam0_3d], eye['time_series'][:,idx_conf], thr=0.5, doplot=False)
# (2) delete diameter velocity outliers
idx_vout, _ = delete_velocity_outliers(eye['time_series'][:,idx_diam0_3d], whsize=10, n_mad=3, doplot=False)
# (3) eyeblink detection (Hershman et al.)
idx_blinks, _ = blink_detection(eye['time_series'][:,idx_diam0_3d], w_smooth=9, doplot=False)

# apply all deletions
diam = eye['time_series'][:,idx_diam0_3d]
diam_corr = diam.copy()
idx_corr = list(idx_conf) + list(idx_vout) + list(idx_blinks)
idx_corr = list(set(idx_corr))
diam_corr[idx_corr] = np.nan
p_corr = 1.*len(idx_corr)/len(diam)
print(f"Total proportion of deleted (NaN) data points: {p_corr:.3f}")

Apply confidence threshold
number of low-confidence data points:  15571
proportion of low-confidence data points: 0.06
Delete diameter velocity outliers (sliding window)
number of detected outliers:  0
proportion of detected outliers: 0.00
Total proportion of deleted (NaN) data points: 0.184
